In [41]:
import pandas as pd
from tqdm.notebook import tqdm
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


## Base Tables

In [42]:
params = {
    'report_start_date': '2023-12-01',
    'report_end_date': '2023-12-31'
}

# Analysis

In [43]:
%%bigquery df_total --project nbcu-ds-sandbox-a-001 --params $params

WITH 
CTE_1 AS (
    SELECT  @report_start_date                                                           AS Report_Month
       --, Account_Type
           ,COUNT(distinct CASE WHEN cohort = 'Email_Targeted' THEN aid END )            AS Distinct_Cohort_Size_Targeted
           ,COUNT(distinct CASE WHEN cohort = 'Holdout' THEN aid END)                    AS Distinct_Cohort_Size_Holdout
           
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN viewer END )                   AS Total_Returns_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN viewer END)                           AS Total_Returns_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Viewing_Time END )              AS Total_Usage_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Viewing_Time END)                      AS Total_Usage_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Repertoire_Pavo_Method END )    AS Total_Repertoire_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Repertoire_Pavo_Method END)            AS Total_Repertoire_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Distinct_Viewing_Sessions END ) AS Total_Viewing_Sessions_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Distinct_Viewing_Sessions END)         AS Total_Viewing_Sessions_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Winback_Denom END )             AS Winback_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Winback_Num END)                AS Winback_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Winback_Denom END )                    AS Winback_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Winback_Num END)                       AS Winback_Num_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Save_Denom END )                AS Save_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Save_Num END)                   AS Save_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Save_Denom END )                       AS Save_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Save_Num END)                          AS Save_Num_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Upgrade_Denom END )             AS Upgrades_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Upgrade_Num END)                AS Upgrades_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Upgrade_Denom END )                    AS Upgrades_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Upgrade_Num END)                       AS Upgrades_Num_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN New_Upgrade_Denom END )         AS Total_New_Upgrade_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN New_Upgrade_Num END)            AS Total_New_Upgrade_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN New_Upgrade_Denom END )                AS Total_New_Upgrade_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN New_Upgrade_Num END)                   AS Total_New_Upgrade_Num_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Paid_Winbacks_Denom END )       AS Total_Paid_Winbacks_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Paid_Winbacks_Num END)          AS Total_Paid_Winbacks_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Paid_Winbacks_Denom END )              AS Total_Paid_Winbacks_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Paid_Winbacks_Num END)                 AS Total_Paid_Winbacks_Num_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Cancel_Save_Denom END )         AS Total_Cancel_Save_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Cancel_Save_Num END)            AS Total_Cancel_Save_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Cancel_Save_Denom END )                AS Total_Cancel_Save_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Cancel_Save_Num END)                   AS Total_Cancel_Save_Num_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN EOM_Paid_Churn_Denom END )      AS EOM_Paid_Churn_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN EOM_Paid_Churn_Num END)         AS EOM_Paid_Churn_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Denom END )             AS EOM_Paid_Churn_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Num END)                AS EOM_Paid_Churn_Num_Holdout
       
       FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v4_Dec22`
       GROUP BY  1 --, 2

), CTE_2 AS (
    SELECT  Report_Month
            -- ,Account_Type
            ,Distinct_Cohort_Size_Targeted                                                            AS Email_Engagers
            ,Distinct_Cohort_Size_Holdout                                                             AS Email_Holdout
            ,safe_divide(Total_Returns_Targeted,Distinct_Cohort_Size_Targeted)                        AS Return_Rate_Engagers
            ,safe_divide(Total_Returns_Holdout,Distinct_Cohort_Size_Holdout)                          AS Return_Rate_Holdout
            ,safe_divide(Total_Usage_Targeted,Distinct_Cohort_Size_Targeted)                          AS Usage_Engagers
            ,safe_divide(Total_Usage_Holdout,Distinct_Cohort_Size_Holdout)                            AS Usage_Holdout
            ,safe_divide(Total_Repertoire_Targeted,Distinct_Cohort_Size_Targeted)                     AS Repertoire_Engagers
            ,safe_divide(Total_Repertoire_Holdout,Distinct_Cohort_Size_Holdout)                       AS Repertoire_Holdout
            ,safe_divide(Total_Viewing_Sessions_Targeted,Distinct_Cohort_Size_Targeted)               AS Sessions_Engagers
            ,safe_divide(Total_Viewing_Sessions_Holdout,Distinct_Cohort_Size_Holdout)                 AS Sessions_Holdout
            ,Winback_Denom_Targeted
            ,safe_divide(Winback_Num_Targeted,Winback_Denom_Targeted)                                 AS Winback_Rate_Engagers
            ,safe_divide(Winback_Num_Holdout,Winback_Denom_Holdout)                                   AS Winback_Rate_Holdout
            ,Save_Denom_Targeted
            ,safe_divide(Save_Num_Targeted,Save_Denom_Targeted)                                       AS Save_Rate_Engagers
            ,safe_divide(Save_Num_Holdout,Save_Denom_Holdout)                                         AS Save_Rate_Holdout
            ,Upgrades_Denom_Targeted
            ,safe_divide(Upgrades_Num_Targeted,Upgrades_Denom_Targeted)                               AS Upgrade_Rate_Engagers
            ,safe_divide(Upgrades_Num_Holdout,Upgrades_Denom_Holdout)                                 AS Upgrade_Rate_Holdout
            ,Total_New_Upgrade_Denom_Targeted
            ,safe_divide(Total_New_Upgrade_Num_Targeted,Total_New_Upgrade_Denom_Targeted)             AS New_Upgrade_Rate_Engagers
            ,safe_divide(Total_New_Upgrade_Num_Holdout,Total_New_Upgrade_Denom_Holdout)               AS New_Upgrade_Rate_Holdout
            ,Total_Paid_Winbacks_Denom_Targeted
            ,safe_divide(Total_Paid_Winbacks_Num_Targeted,Total_Paid_Winbacks_Denom_Targeted)         AS Paid_Winback_Rate_Engagers
            ,safe_divide(Total_Paid_Winbacks_Num_Holdout,Total_Paid_Winbacks_Denom_Holdout)           AS Paid_Winback_Rate_Holdout
            ,Total_Cancel_Save_Denom_Targeted
            ,safe_divide(Total_Cancel_Save_Num_Targeted,Total_Cancel_Save_Denom_Targeted)             AS Cancel_Save_Rate_Engagers
            ,safe_divide(Total_Cancel_Save_Num_Holdout,Total_Cancel_Save_Denom_Holdout)               AS Cancel_Save_Rate_Holdout
            ,EOM_Paid_Churn_Denom_Targeted
            ,safe_divide(EOM_Paid_Churn_Num_Targeted,EOM_Paid_Churn_Denom_Targeted)                   AS Paid_Churn_Rate_Engagers
            ,safe_divide(EOM_Paid_Churn_Num_Holdout,EOM_Paid_Churn_Denom_Holdout)                     AS Paid_Churn_Rate_Holdout
    FROM CTE_1
)

SELECT  Report_Month
       --,Account_Type
       ,Email_Engagers
       ,Email_Holdout

       ,Return_Rate_Engagers                                                                                        AS Return_Rate_Engagers
       ,Return_Rate_Holdout                                                                                         AS Return_Rate_Holdout
       ,Return_Rate_Engagers - Return_Rate_Holdout                                                                  AS Return_Rate_Lift_PTS
       ,safe_divide(Return_Rate_Engagers,Return_Rate_Holdout) *100                                                  AS Return_Rate_Lift_Index
       ,(Return_Rate_Engagers - Return_Rate_Holdout) * Email_Engagers                                               AS Returns_Incrementals

       ,Usage_Engagers                                                                                              AS Usage_Engagers
       ,Usage_Holdout                                                                                               AS Usage_Holdout
       ,Usage_Engagers - Usage_Holdout                                                                              AS Usage_Lift_PTS
       ,safe_divide(Usage_Engagers,Usage_Holdout) *100                                                              AS Usage_Lift_Index
       ,(Usage_Engagers - Usage_Holdout) * Email_Engagers                                                           AS Usage_Incrementals

       ,Repertoire_Engagers                                                                                         AS Repertoire_Engagers
       ,Repertoire_Holdout                                                                                          AS Repertoire_Holdout
       ,Repertoire_Engagers - Repertoire_Holdout                                                                    AS Repertoire_Lift_PTS
       ,safe_divide(Repertoire_Engagers,Repertoire_Holdout) *100                                                    AS Repertoire_Lift_Index
       ,(Repertoire_Engagers - Repertoire_Holdout) * Email_Engagers                                                 AS Repertoire_Incrementals

       ,Sessions_Engagers                                                                                           AS Sessions_Engagers
       ,Sessions_Holdout                                                                                            AS Sessions_Holdout
       ,Sessions_Engagers - Sessions_Holdout                                                                        AS Sessions_Lift_PTS
       ,safe_divide(Sessions_Engagers,Sessions_Holdout) *100                                                        AS Sessions_Lift_Index
       ,(Sessions_Engagers - Sessions_Holdout) * Email_Engagers                                                     AS Sessions_Incrementals

       ,Winback_Rate_Engagers                                                                                       AS Winback_Rate_Engagers
       ,Winback_Rate_Holdout                                                                                        AS Winback_Rate_Holdout
       ,Winback_Rate_Engagers - Winback_Rate_Holdout                                                                AS Winback_Rate_Lift_PTS
       ,safe_divide(Winback_Rate_Engagers,Winback_Rate_Holdout) *100                                                AS Winback_Rate_Lift_Index
       ,(Winback_Rate_Engagers - Winback_Rate_Holdout) * Winback_Denom_Targeted                                     AS Winback_Incrementals

       ,Save_Rate_Engagers                                                                                          AS Save_Rate_Engagers
       ,Save_Rate_Holdout                                                                                           AS Save_Rate_Holdout
       ,Save_Rate_Engagers - Save_Rate_Holdout                                                                      AS Save_Rate_Lift_PTS
       ,safe_divide(Save_Rate_Engagers,Save_Rate_Holdout) *100                                                      AS Save_Rate_Lift_Index
       ,(Save_Rate_Engagers - Save_Rate_Holdout) * Save_Denom_Targeted                                              AS Save_Rate_Lift_Incrementals

       ,Upgrade_Rate_Engagers                                                                                       AS Upgrade_Rate_Engagers
       ,Upgrade_Rate_Holdout                                                                                        AS Upgrade_Rate_Holdout
       ,Upgrade_Rate_Engagers - Upgrade_Rate_Holdout                                                                AS Upgrade_Rate_Lift_PTS
       ,safe_divide(Upgrade_Rate_Engagers,Upgrade_Rate_Holdout) *100                                                AS Upgrade_Rate_Lift_Index
       ,(Upgrade_Rate_Engagers - Upgrade_Rate_Holdout) * Upgrades_Denom_Targeted                                    AS Upgrade_Incrementals

       ,New_Upgrade_Rate_Engagers                                                                                   AS New_Upgrade_Rate_Engagers
       ,New_Upgrade_Rate_Holdout                                                                                    AS New_Upgrade_Rate_Holdout
       ,New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout                                                        AS New_Upgrade_Rate_Lift_PTS
       ,safe_divide(New_Upgrade_Rate_Engagers,New_Upgrade_Rate_Holdout) *100                                        AS New_Upgrade_Rate_Lift_Index
       ,(New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout) * Total_New_Upgrade_Denom_Targeted                   AS New_Upgrade_Incrementals

       ,Paid_Winback_Rate_Engagers                                                                                  AS Paid_Winback_Rate_Engagers
       ,Paid_Winback_Rate_Holdout                                                                                   AS Paid_Winback_Rate_Holdout
       ,Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout                                                      AS Paid_Winback_Rate_Lift_PTS
       ,safe_divide(Paid_Winback_Rate_Engagers,Paid_Winback_Rate_Holdout) *100                                      AS Paid_Winback_Rate_Lift_Index
       ,(Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout) * Total_Paid_Winbacks_Denom_Targeted               AS Paid_Winback_Rate_Lift_Incrementals

       ,Cancel_Save_Rate_Engagers                                                                             AS Cancel_Save_Rate_Engagers
       ,Cancel_Save_Rate_Holdout                                                                              AS Cancel_Save_Rate_Holdout
       ,Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout                                            AS Cancel_Save_Rate_Lift_PTS
       ,safe_divide(Cancel_Save_Rate_Engagers,Cancel_Save_Rate_Holdout) *100                            AS Cancel_Save_Rate_Lift_Index
       ,(Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout) * Total_Cancel_Save_Denom_Targeted     AS Cancel_Save_Rate_Incrementals

       ,Paid_Churn_Rate_Engagers                                                                                    AS EOM_Paid_Churn_Rate_Engagers
       ,Paid_Churn_Rate_Holdout                                                                                     AS EOM_Paid_Churn_Rate_Holdout
       ,Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                                                          AS EOM_Paid_Churn_Rate_Lift_PTS
       ,safe_divide(Paid_Churn_Rate_Engagers,Paid_Churn_Rate_Holdout) *100                                          AS EOM_Paid_Churn_Rate_Lift_Index
       ,(Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted                        AS EOM_Paid_Churn_Rate_Incrementals

FROM CTE_2

Query is running:   0%|          |

Downloading:   0%|          |

In [44]:
df_total

,Report_Month,Email_Engagers,Email_Holdout,Return_Rate_Engagers,Return_Rate_Holdout,Return_Rate_Lift_PTS,Return_Rate_Lift_Index,Returns_Incrementals,Usage_Engagers,Usage_Holdout,...,Cancel_Save_Rate_Engagers,Cancel_Save_Rate_Holdout,Cancel_Save_Rate_Lift_PTS,Cancel_Save_Rate_Lift_Index,Cancel_Save_Rate_Incrementals,EOM_Paid_Churn_Rate_Engagers,EOM_Paid_Churn_Rate_Holdout,EOM_Paid_Churn_Rate_Lift_PTS,EOM_Paid_Churn_Rate_Lift_Index,EOM_Paid_Churn_Rate_Incrementals
0,2023-12-01,37394831,2173801,0.46068,0.389008,0.071672,118.424251,2.680155e+06,8.964284,7.244629,...,0.021096,0.02228,-0.001184,94.683863,-555.454133,0.073033,0.073885,-0.000852,98.847228,-9564.358394


In [45]:
%%bigquery df_acc --project nbcu-ds-sandbox-a-001 --params $params

WITH 
CTE_1 AS (
    SELECT  @report_start_date                                                           AS Report_Month
            , Account_Type
           ,COUNT(distinct CASE WHEN cohort = 'Email_Targeted' THEN aid END )            AS Distinct_Cohort_Size_Targeted
           ,COUNT(distinct CASE WHEN cohort = 'Holdout' THEN aid END)                    AS Distinct_Cohort_Size_Holdout
           
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN viewer END )                   AS Total_Returns_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN viewer END)                           AS Total_Returns_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Viewing_Time END )              AS Total_Usage_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Viewing_Time END)                      AS Total_Usage_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Repertoire_Pavo_Method END )    AS Total_Repertoire_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Repertoire_Pavo_Method END)            AS Total_Repertoire_Holdout
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Distinct_Viewing_Sessions END ) AS Total_Viewing_Sessions_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Distinct_Viewing_Sessions END)         AS Total_Viewing_Sessions_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Winback_Denom END )             AS Winback_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Winback_Num END)                AS Winback_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Winback_Denom END )                    AS Winback_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Winback_Num END)                       AS Winback_Num_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Save_Denom END )                AS Save_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Save_Num END)                   AS Save_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Save_Denom END )                       AS Save_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Save_Num END)                          AS Save_Num_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Upgrade_Denom END )             AS Upgrades_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Upgrade_Num END)                AS Upgrades_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Upgrade_Denom END )                    AS Upgrades_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Upgrade_Num END)                       AS Upgrades_Num_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN New_Upgrade_Denom END )         AS Total_New_Upgrade_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN New_Upgrade_Num END)            AS Total_New_Upgrade_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN New_Upgrade_Denom END )                AS Total_New_Upgrade_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN New_Upgrade_Num END)                   AS Total_New_Upgrade_Num_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Paid_Winbacks_Denom END )       AS Total_Paid_Winbacks_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Paid_Winbacks_Num END)          AS Total_Paid_Winbacks_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Paid_Winbacks_Denom END )              AS Total_Paid_Winbacks_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Paid_Winbacks_Num END)                 AS Total_Paid_Winbacks_Num_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Cancel_Save_Denom END )         AS Total_Cancel_Save_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN Cancel_Save_Num END)            AS Total_Cancel_Save_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Cancel_Save_Denom END )                AS Total_Cancel_Save_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN Cancel_Save_Num END)                   AS Total_Cancel_Save_Num_Holdout

           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN EOM_Paid_Churn_Denom END )      AS EOM_Paid_Churn_Denom_Targeted
           ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN EOM_Paid_Churn_Num END)         AS EOM_Paid_Churn_Num_Targeted
           ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Denom END )             AS EOM_Paid_Churn_Denom_Holdout
           ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Num END)                AS EOM_Paid_Churn_Num_Holdout
       
       FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v4_Dec22`
       GROUP BY  1, 2

), CTE_2 AS (
    SELECT  Report_Month
            ,Account_Type
            ,Distinct_Cohort_Size_Targeted                                                            AS Email_Engagers
            ,Distinct_Cohort_Size_Holdout                                                             AS Email_Holdout
            ,safe_divide(Total_Returns_Targeted,Distinct_Cohort_Size_Targeted)                        AS Return_Rate_Engagers
            ,safe_divide(Total_Returns_Holdout,Distinct_Cohort_Size_Holdout)                          AS Return_Rate_Holdout
            ,safe_divide(Total_Usage_Targeted,Distinct_Cohort_Size_Targeted)                          AS Usage_Engagers
            ,safe_divide(Total_Usage_Holdout,Distinct_Cohort_Size_Holdout)                            AS Usage_Holdout
            ,safe_divide(Total_Repertoire_Targeted,Distinct_Cohort_Size_Targeted)                     AS Repertoire_Engagers
            ,safe_divide(Total_Repertoire_Holdout,Distinct_Cohort_Size_Holdout)                       AS Repertoire_Holdout
            ,safe_divide(Total_Viewing_Sessions_Targeted,Distinct_Cohort_Size_Targeted)               AS Sessions_Engagers
            ,safe_divide(Total_Viewing_Sessions_Holdout,Distinct_Cohort_Size_Holdout)                 AS Sessions_Holdout
            ,Winback_Denom_Targeted
            ,safe_divide(Winback_Num_Targeted,Winback_Denom_Targeted)                                 AS Winback_Rate_Engagers
            ,safe_divide(Winback_Num_Holdout,Winback_Denom_Holdout)                                   AS Winback_Rate_Holdout
            ,Save_Denom_Targeted
            ,safe_divide(Save_Num_Targeted,Save_Denom_Targeted)                                       AS Save_Rate_Engagers
            ,safe_divide(Save_Num_Holdout,Save_Denom_Holdout)                                         AS Save_Rate_Holdout
            ,Upgrades_Denom_Targeted
            ,safe_divide(Upgrades_Num_Targeted,Upgrades_Denom_Targeted)                               AS Upgrade_Rate_Engagers
            ,safe_divide(Upgrades_Num_Holdout,Upgrades_Denom_Holdout)                                 AS Upgrade_Rate_Holdout
            ,Total_New_Upgrade_Denom_Targeted
            ,safe_divide(Total_New_Upgrade_Num_Targeted,Total_New_Upgrade_Denom_Targeted)             AS New_Upgrade_Rate_Engagers
            ,safe_divide(Total_New_Upgrade_Num_Holdout,Total_New_Upgrade_Denom_Holdout)               AS New_Upgrade_Rate_Holdout
            ,Total_Paid_Winbacks_Denom_Targeted
            ,safe_divide(Total_Paid_Winbacks_Num_Targeted,Total_Paid_Winbacks_Denom_Targeted)         AS Paid_Winback_Rate_Engagers
            ,safe_divide(Total_Paid_Winbacks_Num_Holdout,Total_Paid_Winbacks_Denom_Holdout)           AS Paid_Winback_Rate_Holdout
            ,Total_Cancel_Save_Denom_Targeted
            ,safe_divide(Total_Cancel_Save_Num_Targeted,Total_Cancel_Save_Denom_Targeted)             AS Cancel_Save_Rate_Engagers
            ,safe_divide(Total_Cancel_Save_Num_Holdout,Total_Cancel_Save_Denom_Holdout)               AS Cancel_Save_Rate_Holdout
            ,EOM_Paid_Churn_Denom_Targeted
            ,safe_divide(EOM_Paid_Churn_Num_Targeted,EOM_Paid_Churn_Denom_Targeted)                   AS Paid_Churn_Rate_Engagers
            ,safe_divide(EOM_Paid_Churn_Num_Holdout,EOM_Paid_Churn_Denom_Holdout)                     AS Paid_Churn_Rate_Holdout
    FROM CTE_1
)

SELECT  Report_Month
       ,Account_Type
       ,Email_Engagers
       ,Email_Holdout

       ,Return_Rate_Engagers                                                                                        AS Return_Rate_Engagers
       ,Return_Rate_Holdout                                                                                         AS Return_Rate_Holdout
       ,Return_Rate_Engagers - Return_Rate_Holdout                                                                  AS Return_Rate_Lift_PTS
       ,safe_divide(Return_Rate_Engagers,Return_Rate_Holdout) *100                                                  AS Return_Rate_Lift_Index
       ,(Return_Rate_Engagers - Return_Rate_Holdout) * Email_Engagers                                               AS Returns_Incrementals

       ,Usage_Engagers                                                                                              AS Usage_Engagers
       ,Usage_Holdout                                                                                               AS Usage_Holdout
       ,Usage_Engagers - Usage_Holdout                                                                              AS Usage_Lift_PTS
       ,safe_divide(Usage_Engagers,Usage_Holdout) *100                                                              AS Usage_Lift_Index
       ,(Usage_Engagers - Usage_Holdout) * Email_Engagers                                                           AS Usage_Incrementals

       ,Repertoire_Engagers                                                                                         AS Repertoire_Engagers
       ,Repertoire_Holdout                                                                                          AS Repertoire_Holdout
       ,Repertoire_Engagers - Repertoire_Holdout                                                                    AS Repertoire_Lift_PTS
       ,safe_divide(Repertoire_Engagers,Repertoire_Holdout) *100                                                    AS Repertoire_Lift_Index
       ,(Repertoire_Engagers - Repertoire_Holdout) * Email_Engagers                                                 AS Repertoire_Incrementals

       ,Sessions_Engagers                                                                                           AS Sessions_Engagers
       ,Sessions_Holdout                                                                                            AS Sessions_Holdout
       ,Sessions_Engagers - Sessions_Holdout                                                                        AS Sessions_Lift_PTS
       ,safe_divide(Sessions_Engagers,Sessions_Holdout) *100                                                        AS Sessions_Lift_Index
       ,(Sessions_Engagers - Sessions_Holdout) * Email_Engagers                                                     AS Sessions_Incrementals

       ,Winback_Rate_Engagers                                                                                       AS Winback_Rate_Engagers
       ,Winback_Rate_Holdout                                                                                        AS Winback_Rate_Holdout
       ,Winback_Rate_Engagers - Winback_Rate_Holdout                                                                AS Winback_Rate_Lift_PTS
       ,safe_divide(Winback_Rate_Engagers,Winback_Rate_Holdout) *100                                                AS Winback_Rate_Lift_Index
       ,(Winback_Rate_Engagers - Winback_Rate_Holdout) * Winback_Denom_Targeted                                     AS Winback_Incrementals

       ,Save_Rate_Engagers                                                                                          AS Save_Rate_Engagers
       ,Save_Rate_Holdout                                                                                           AS Save_Rate_Holdout
       ,Save_Rate_Engagers - Save_Rate_Holdout                                                                      AS Save_Rate_Lift_PTS
       ,safe_divide(Save_Rate_Engagers,Save_Rate_Holdout) *100                                                      AS Save_Rate_Lift_Index
       ,(Save_Rate_Engagers - Save_Rate_Holdout) * Save_Denom_Targeted                                              AS Save_Rate_Lift_Incrementals

       ,Upgrade_Rate_Engagers                                                                                       AS Upgrade_Rate_Engagers
       ,Upgrade_Rate_Holdout                                                                                        AS Upgrade_Rate_Holdout
       ,Upgrade_Rate_Engagers - Upgrade_Rate_Holdout                                                                AS Upgrade_Rate_Lift_PTS
       ,safe_divide(Upgrade_Rate_Engagers,Upgrade_Rate_Holdout) *100                                                AS Upgrade_Rate_Lift_Index
       ,(Upgrade_Rate_Engagers - Upgrade_Rate_Holdout) * Upgrades_Denom_Targeted                                    AS Upgrade_Incrementals

       ,New_Upgrade_Rate_Engagers                                                                                   AS New_Upgrade_Rate_Engagers
       ,New_Upgrade_Rate_Holdout                                                                                    AS New_Upgrade_Rate_Holdout
       ,New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout                                                        AS New_Upgrade_Rate_Lift_PTS
       ,safe_divide(New_Upgrade_Rate_Engagers,New_Upgrade_Rate_Holdout) *100                                        AS New_Upgrade_Rate_Lift_Index
       ,(New_Upgrade_Rate_Engagers - New_Upgrade_Rate_Holdout) * Total_New_Upgrade_Denom_Targeted                   AS New_Upgrade_Incrementals

       ,Paid_Winback_Rate_Engagers                                                                                  AS Paid_Winback_Rate_Engagers
       ,Paid_Winback_Rate_Holdout                                                                                   AS Paid_Winback_Rate_Holdout
       ,Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout                                                      AS Paid_Winback_Rate_Lift_PTS
       ,safe_divide(Paid_Winback_Rate_Engagers,Paid_Winback_Rate_Holdout) *100                                      AS Paid_Winback_Rate_Lift_Index
       ,(Paid_Winback_Rate_Engagers - Paid_Winback_Rate_Holdout) * Total_Paid_Winbacks_Denom_Targeted               AS Paid_Winback_Rate_Lift_Incrementals

       ,Cancel_Save_Rate_Engagers                                                                             AS Cancel_Save_Rate_Engagers
       ,Cancel_Save_Rate_Holdout                                                                              AS Cancel_Save_Rate_Holdout
       ,Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout                                            AS Cancel_Save_Rate_Lift_PTS
       ,safe_divide(Cancel_Save_Rate_Engagers,Cancel_Save_Rate_Holdout) *100                            AS Cancel_Save_Rate_Lift_Index
       ,(Cancel_Save_Rate_Engagers - Cancel_Save_Rate_Holdout) * Total_Cancel_Save_Denom_Targeted     AS Cancel_Save_Rate_Incrementals

       ,Paid_Churn_Rate_Engagers                                                                                    AS EOM_Paid_Churn_Rate_Engagers
       ,Paid_Churn_Rate_Holdout                                                                                     AS EOM_Paid_Churn_Rate_Holdout
       ,Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                                                          AS EOM_Paid_Churn_Rate_Lift_PTS
       ,safe_divide(Paid_Churn_Rate_Engagers,Paid_Churn_Rate_Holdout) *100                                          AS EOM_Paid_Churn_Rate_Lift_Index
       ,(Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted                        AS EOM_Paid_Churn_Rate_Incrementals

FROM CTE_2

Query is running:   0%|          |

Downloading:   0%|          |

In [46]:
df_acc = df_acc.sort_values(by='Account_Type')
df_acc

,Report_Month,Account_Type,Email_Engagers,Email_Holdout,Return_Rate_Engagers,Return_Rate_Holdout,Return_Rate_Lift_PTS,Return_Rate_Lift_Index,Returns_Incrementals,Usage_Engagers,...,Cancel_Save_Rate_Engagers,Cancel_Save_Rate_Holdout,Cancel_Save_Rate_Lift_PTS,Cancel_Save_Rate_Lift_Index,Cancel_Save_Rate_Incrementals,EOM_Paid_Churn_Rate_Engagers,EOM_Paid_Churn_Rate_Holdout,EOM_Paid_Churn_Rate_Lift_PTS,EOM_Paid_Churn_Rate_Lift_Index,EOM_Paid_Churn_Rate_Incrementals
0,2023-12-01,Bundled Premium,7686413,415656,0.572315,0.528678,0.043638,108.254088,335415.927777,8.740496,...,0.009491,0.005051,0.004441,187.928936,18.247475,NaN,NaN,NaN,NaN,NaN
2,2023-12-01,Free,18479022,1233505,0.174047,0.151053,0.022994,115.222553,424909.758359,0.666495,...,0.000601,0.000528,0.000074,113.983073,18.156159,NaN,NaN,NaN,NaN,NaN
1,2023-12-01,Paying SVOD,11229396,524640,0.855947,0.837818,0.018129,102.163788,203573.381542,22.772254,...,0.044375,0.046768,-0.002393,94.882725,-523.470123,0.073033,0.073885,-0.000852,98.847228,-9564.358394


In [47]:
df_total.to_excel('email_kpi_tot_v4_Dec22_qualifier_report_end.xlsx', index=False)
df_acc.to_excel('email_kpi_acc_v4_Dec22_qualifier_report_end.xlsx', index=False)

# Churn Breakdown

In [48]:
%%bigquery df_churn --project nbcu-ds-sandbox-a-001 --params $params

WITH 
CTE_1 AS (
  SELECT 
    @report_start_date AS Report_Month
    --, Account_Type
    , tenure_paid_lens
      
    ,COUNT(distinct CASE WHEN (cohort = 'Email_Targeted') AND (Account_Type = 'Paying SVOD') THEN aid END )            AS Distinct_Cohort_Size_Targeted
    ,COUNT(distinct CASE WHEN (cohort = 'Holdout') AND (Account_Type = 'Paying SVOD') THEN aid END)                    AS Distinct_Cohort_Size_Holdout

    ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN EOM_Paid_Churn_Denom END )      AS EOM_Paid_Churn_Denom_Targeted
    ,SUM(CASE WHEN cohort = 'Email_Targeted' THEN EOM_Paid_Churn_Num END)         AS EOM_Paid_Churn_Num_Targeted
    ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Denom END )             AS EOM_Paid_Churn_Denom_Holdout
    ,SUM(CASE WHEN cohort = 'Holdout' THEN EOM_Paid_Churn_Num END)                AS EOM_Paid_Churn_Num_Holdout
  FROM `nbcu-ds-sandbox-a-001.SLi_sandbox.CRM_KPI_Base_v4_Dec22`
  GROUP BY 1,2
), CTE_2 AS (
    SELECT Report_Month
  --, Account_Type
  , account_tenure
  , Distinct_Cohort_Size_Targeted   as Email_Engagers
  , Distinct_Cohort_Size_Holdout   as Email_Holdout

  , EOM_Paid_Churn_Denom_Targeted
  , safe_divide(EOM_Paid_Churn_Num_Targeted, EOM_Paid_Churn_Denom_Targeted) as Paid_Churn_Rate_Engagers
  , safe_divide(EOM_Paid_Churn_Num_Holdout, EOM_Paid_Churn_Denom_Holdout) as Paid_Churn_Rate_Holdout

  from CTE_1
)

SELECT Report_Month
--, Account_Type
, tenure_paid_lens
, Email_Engagers
, Email_Holdout

, Paid_Churn_Rate_Engagers                                                  as EOM_Paid_Churn_Rate_Engagers
, Paid_Churn_Rate_Holdout                                                   as EOM_Paid_Churn_Rate_Holdout
, Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout                        as EOM_Paid_Churn_Rate_Lift_PTS
, safe_divide(Paid_Churn_Rate_Engagers, Paid_Churn_Rate_Holdout) *100                   as EOM_Paid_Churn_Rate_Lift_Index
, (Paid_Churn_Rate_Engagers - Paid_Churn_Rate_Holdout) * EOM_Paid_Churn_Denom_Targeted as EOM_Paid_Churn_Rate_Incrementals

FROM CTE_2
order by 1,2

Executing query with job ID: c5ab939b-5cbb-4ca7-be19-3081ab453496
Query executing: 0.54s


ERROR:
 400 Unrecognized name: tenure_paid_lens at [6:7]

Location: US
Job ID: c5ab939b-5cbb-4ca7-be19-3081ab453496



In [49]:
df_churn

QueryJob<project=nbcu-ds-sandbox-a-001, location=US, id=c5ab939b-5cbb-4ca7-be19-3081ab453496>

In [50]:
df_churn.to_excel('email_kpi_v4_Dec22_churn.xlsx')

AttributeError: 'QueryJob' object has no attribute 'to_excel'